# Pachyderm-Seldon Integration: Version Controlled Models

[Pachyderm](https://pachyderm.io/) is a data science platform that combines Data Lineage with End-to-End Pipelines. [Seldon-Core](https://www.seldon.io/tech/products/core/) is an open-source platform for rapidly deploying machine learning models.

This notebook will show you how to perform data-driven model training using Pachyderm and deploy that model using Seldon-Core. This represents the simplest integration between Pachyderm and Seldon.

### Highlights

The highlight in this example is the use of Pachyderm's S3 gateway, where version controlled pachyderm repositories are exposed via an S3-like API. This means that Seldon-Core's pre-packaged model servers, which typically read from blob storage, can read directly from Pachyderm.

This implements the machine learning best practice of provenance. You can trace the model artifact's lineage right back to the version of the data that it was trained on, without making any changes to your usual way of working.

![overview image](./images/pachyderm-simple-demo.png)

## Prerequisites

 * A kubernetes cluster with kubectl configured
 * [A seldon-core cluster installed and configured](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html)
 * `curl`
 * Pachyderm's `pachctl` binary installed following the [official documentation](https://docs.pachyderm.com/latest/getting_started/local_installation/#install-pachctl)

Pachyderm is controlled and deployed via the `pachctl`. This document assumes that you have version `1.11.8` installed, although it should work with any `1.x.x` version.

In [22]:
!pachctl version --client-only
!kubectl get po -A

1.11.8
NAMESPACE            NAME                                          READY   STATUS    RESTARTS   AGE
kube-system          coredns-f9fd979d6-9grz7                       1/1     Running   0          9m10s
kube-system          coredns-f9fd979d6-bdx46                       1/1     Running   0          9m10s
kube-system          etcd-kind-control-plane                       1/1     Running   0          9m15s
kube-system          kindnet-qnqmf                                 1/1     Running   0          9m10s
kube-system          kube-apiserver-kind-control-plane             1/1     Running   0          9m15s
kube-system          kube-controller-manager-kind-control-plane    1/1     Running   2          9m15s
kube-system          kube-proxy-n2cwg                              1/1     Running   0          9m10s
kube-system          kube-scheduler-kind-control-plane             1/1     Running   2          9m15s
kube-system          seldon-spartakus-volunteer-5b57b95596-jv49d   1/1     Ru

## 1. Installing Pachyderm

Use `pachctl` to deploy Pachyderm to your cluster. In the command below I disable the pachyderm dashboard because it is not used in this example and I intentionally disable sharing of the docker socket to reduce the chance that rogue containers can access my system.

In [2]:
!kubectl create ns pachyderm
!pachctl deploy local --no-expose-docker-socket --no-dashboard --namespace pachyderm

namespace/pachyderm created
serviceaccount/pachyderm created
serviceaccount/pachyderm-worker created
clusterrole.rbac.authorization.k8s.io/pachyderm configured
clusterrolebinding.rbac.authorization.k8s.io/pachyderm configured
role.rbac.authorization.k8s.io/pachyderm-worker created
rolebinding.rbac.authorization.k8s.io/pachyderm-worker created
deployment.apps/etcd created
service/etcd created
service/pachd created
service/pachd-peer created
deployment.apps/pachd created
secret/pachyderm-storage-secret created

Pachyderm is launching. Check its status with "kubectl get all"



In [3]:
!kubectl rollout status deployment -n pachyderm pachd

Waiting for deployment "pachd" rollout to finish: 0 out of 1 new replicas have been updated...
Waiting for deployment "pachd" rollout to finish: 0 of 1 updated replicas are available...
deployment "pachd" successfully rolled out


## 2. Add the Training Data

> Note: If you are using a non-standard Pachyderm namespace, you will need to run `pachctl port-forward` in a separate terminal

Before we train a model, I will first create a new Pachyderm repository to version control the data, then commit the training data to that repository. You should be able to see `iris.csv` in the `master` branch of the repository `training`.

In [4]:
!pachctl create repo training

In [5]:
!cd data && pachctl put file training@master -f iris.csv

iris.csv: 4.44 KiB / 4.44 KiB [===============================] 100.00% ? p/s 0s


In [6]:
!pachctl list repo

NAME     CREATED       SIZE (MASTER) DESCRIPTION 
training 2 seconds ago 4.443KiB                  


In [7]:
!pachctl list file training@master

NAME      TYPE SIZE     
/iris.csv file 4.443KiB 


## 3. Train the Model Using a Pachyderm Pipeline

Pachyderm Pipelines are the computational component of the Pachyderm platform. They are responsible for reading data from a specified source, such as a Pachyderm repo, transforming it according to the pipeline configuration, and writing the result to an output repo. Pachyderm pipelines can be easily chained together to create a directed acyclic graph (DAG).

I define a simple pipeline that is inspired by a [similar Pachyderm example](https://github.com/pachyderm/pachyderm/tree/master/examples/ml/iris). It accepts training data, trains a logistic classifier, and returns a pickled model in joblib format.

This uses a container built from the [iris-train-python-svm](./iris-train-python-svm) directory to perform the training, but you could use your own. The input is defined to be the `training` Pachyderm repository and the output will be stored in a repository with the same name as the pipeline (`model`). Remember to push the container to somewhere that the cluster can acccess. For example, you can use the `make kind_load` target in the [Makefile](./iris-train-python-svm/Makefile) to push the container to a `kind` cluster.

In [8]:
%%writefile train.json
{
  "pipeline": {
    "name": "model"
  },
  "description": "A pipeline that trains a SVM model based on the data in the `data` directory by using a Python script.",
  "transform": {
    "image": "seldonio/pachyderm-iris-trainer:1.0.0",
    "cmd": [
      "python3",
      "/code/pytrain.py",
      "/pfs/training/",
      "/pfs/out/"
    ]
  },
  "parallelism_spec": {
    "constant": "1"
  },
  "input": {
    "pfs": {
      "repo": "training",
      "glob": "/"
    }
  }
}

Overwriting train.json


In [9]:
!pachctl create pipeline -f train.json

In [10]:
!pachctl list pipeline

NAME  VERSION INPUT      CREATED       STATE / LAST JOB   DESCRIPTION                                                                                            
model 1       training:/ 3 seconds ago running / starting A pipeline that trains a SVM model based on the data in the `data` directory by using a Python script. 


## 4. Verify Pipeline Success

It may take a while for the job to start the first time, as Kubernetes pulls the container. But eventually you should see that the job completes and a new file is committed to the `master` branch of the `model` repository.

In [14]:
!pachctl list job

ID                               PIPELINE STARTED       DURATION  RESTART PROGRESS  DL       UL       STATE   
43221c6800ff49f6867f578c8e202114 model    7 seconds ago 5 seconds 0       1 + 0 / 1 4.443KiB 1.193KiB success 


In [15]:
!pachctl list file model@master

NAME          TYPE SIZE     
/model.joblib file 1.193KiB 


## 5. Deploy Seldon-Core Powered Sklearn Server

Now that you have a model artefact, you can use Pachyderm's S3 gateway feature to directly load the version controlled model into a Seldon-Core deployment.

The key is to create a container secret that points to the pachyderm cluster on the right port. If you have installed Pachyderm by following this tutorial, then it will have installed with no credentials. In this case the corresponding access key and secret are not used (but must be set). The endpoint URL is pointing to the in-cluster FQDN of the pachyderm service on port `600`.

Next you can deploy your sklearn-based Seldon Deployment in the usual mannor, using the following format for the S3 bucket: `s3://${branch}.${repository}/${directory}`. Since our model is written to the root of the directory, the final s3 `modelUri` is `s3://master.model`. 

In [16]:
%%writefile secret.yaml

apiVersion: v1
kind: Secret
metadata:
  name: seldon-init-container-secret
type: Opaque
stringData:
  AWS_ACCESS_KEY_ID: noauth
  AWS_SECRET_ACCESS_KEY: noauth
  AWS_ENDPOINT_URL: http://pachd.pachyderm.svc.cluster.local:600
  USE_SSL: "false"


Overwriting secret.yaml


In [17]:
!kubectl -n seldon apply -f secret.yaml

secret/seldon-init-container-secret created


In [18]:
%%writefile deploy.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: pachyderm-sklearn
spec:
  annotations:
    seldon.io/executor: "true"
  name: iris
  predictors:
  - componentSpecs:
    graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: s3://master.model
      envSecretRefName: seldon-init-container-secret
      name: classifier
    name: default
    replicas: 1

Overwriting deploy.yaml


In [19]:
!kubectl -n seldon apply -f deploy.yaml

seldondeployment.machinelearning.seldon.io/pachyderm-sklearn created


In [20]:
!kubectl -n seldon rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=pachyderm-sklearn -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "pachyderm-sklearn-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "pachyderm-sklearn-default-0-classifier" successfully rolled out


## 6. Test the Deployment

Now that the deployment has completed, you can call the model predict endpoint via the port-forwarded ingress FDQN and the seldon-core API. The input matches the convension expected by the mdoel (set by the training data) and results in predicted class probabilities.

In [24]:
!curl -s -X POST -H 'Content-Type: application/json' \
    -d '{"data":{"ndarray":[[5.964, 4.006, 2.081, 1.031]]}}' \
    http://localhost:8003/seldon/seldon/pachyderm-sklearn/api/v1.0/predictions  | jq .

{
  "data": {
    "names": [
      "t:0",
      "t:1",
      "t:2"
    ],
    "ndarray": [
      [
        0.9550249833404882,
        0.04491709964435876,
        5.791701515296898e-05
      ]
    ]
  },
  "meta": {
    "requestPath": {
      "classifier": "seldonio/sklearnserver:1.5.1"
    }
  }
}


## 7. Bonus: Rebuilding the Model

The beauty of having models version controlled is that when you change the data, you can automatically retrain the model.

Below I'm overwriting the old training data with a slightly smaller version of the same. When I do this, the pipeline notices and automatically runs. That means that the model artifact is regenerated and place back in it's repo.

At the end, all we need to do is restart the pod, to get Seldon-Core to reload model from it's S3 endpoint. You'll see in the final call of the predict endpoint that the result has changed, even though we're passing in the same input data. This is because the decision boundaries have changed slightly due to the different training data distribution.


In another example I will show you how to automate this.

In [25]:
!cd data && pachctl put file training@master:/iris.csv -f iris_2.csv

iris_2.csv: 1.29 KiB / 1.29 KiB [=============================] 100.00% ? p/s 0s


In [26]:
!pachctl list file training@master

NAME      TYPE SIZE     
/iris.csv file 5.729KiB 


In [28]:
!pachctl list job

ID                               PIPELINE STARTED       DURATION  RESTART PROGRESS  DL       UL       STATE   
cac7add54e9f451f8fdf17360efbcbea model    9 seconds ago 4 seconds 0       1 + 0 / 1 5.729KiB 1.193KiB success 
43221c6800ff49f6867f578c8e202114 model    7 minutes ago 5 seconds 0       1 + 0 / 1 4.443KiB 1.193KiB success 


In [31]:
!kubectl -n seldon delete -f deploy.yaml
!kubectl -n seldon apply -f deploy.yaml
!sleep 1
!kubectl -n seldon rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=pachyderm-sklearn -o jsonpath='{.items[0].metadata.name}')

seldondeployment.machinelearning.seldon.io "pachyderm-sklearn" deleted
seldondeployment.machinelearning.seldon.io/pachyderm-sklearn created
Waiting for deployment "pachyderm-sklearn-default-0-classifier" rollout to finish: 0 out of 1 new replicas have been updated...
Waiting for deployment "pachyderm-sklearn-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "pachyderm-sklearn-default-0-classifier" successfully rolled out


In [34]:
!curl -s -X POST -H 'Content-Type: application/json' \
    -d '{"data":{"ndarray":[[5.964, 4.006, 2.081, 1.031]]}}' \
    http://localhost:8003/seldon/seldon/pachyderm-sklearn/api/v1.0/predictions | jq .

{
  "data": {
    "names": [
      "t:0",
      "t:1",
      "t:2"
    ],
    "ndarray": [
      [
        0.9622688698661227,
        0.03771194327740441,
        1.9186856472893013e-05
      ]
    ]
  },
  "meta": {
    "requestPath": {
      "classifier": "seldonio/sklearnserver:1.5.1"
    }
  }
}


## Cleanup

In [ ]:
!kubectl -n seldon delete -f deploy.yaml